In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from more_itertools import flatten
import scipy as sp
from itertools import groupby
import numpy as np
import re
from collections import Counter
from sys import maxint
from shuyo.vocabulary import load_file, load_corpus, Vocabulary
from shuyo.hdplda2 import HDPLDA
from pprint import pprint
import matplotlib.pyplot as plt
from random import shuffle
import pandas as pd
minint = -maxint - 1

In [4]:
%matplotlib inline

## Building the dataset

[Teh et al](http://www.cs.berkeley.edu/%7Ejordan/papers/hierarchical-dp.pdf) run their HDP-LDA sampler on the "Nematode biology abstracts" corpus. These abstracts are no longer available where linked in the paper, but I found them [here](https://web.archive.org/web/20040328153507/http://elegans.swmed.edu/wli/cgcbib).

The authors don't provide their preprocessed data. They say:

> There are 5838 abstracts in total. After removing standard stopwords and words appearing less than 10 times, we are left with 476441 words in total and a vocabulary size of 5699.

Below, I process the data and get 5828 abstracts (by removing duplicated citations. Using the stopwords provided by scikit-learn and eliminating words that occur fewer than 10 times, I got 5800 terms and a total of 488903 words.

### Attempting to replicate Teh's preprocessing

In [63]:
with open("../data/nematode biology abstracts.txt", "r") as f:
    raw_abstracts = f.read()

def vectorize(docs, *args, **qsargs):
    vectorizer = CountVectorizer(stop_words='english',  *args, **qsargs)
    data = vectorizer.fit_transform(docs)
    col_freq_filter = np.asarray(data.sum(axis=0) > 9)[0]
    print "col_freq_filter", Counter(col_freq_filter).most_common()
    print "total words", data[:, col_freq_filter].sum()
    vocabulary = [t for _, t in sorted([(v, k) for k, v in vectorizer.vocabulary_.iteritems()])]
    num_docs = len(docs)
    vectorized_docs = [[] for _ in docs] 
    for row, col, count in zip(*sp.sparse.find(data)):
        if not col_freq_filter[col]:
            continue
        for word in range(count):
            vectorized_docs[row].append(vocabulary[col])
    return vectorized_docs

processed_abstracts = {}
for _ in re.findall("Citation: (.+?)----", raw_abstracts, re.DOTALL):
    citation = _.split("\n")[0]
    _ = re.findall("Abstract(.+)", _, re.DOTALL)[0]
    _ = _[2:]
    
    _ = re.sub("\n", " ", _)
    _ = re.sub("[ ]{2,}", " ", _)
    _ = _.strip()
    processed_abstracts[citation] = _

print "number of abstracts", len(processed_abstracts)
docs = vectorize(processed_abstracts.values())

number of abstracts 5828
col_freq_filter [(False, 19283), (True, 5800)]
total words 488903


In [64]:
source_file = "preprocessed_bio_abstracts.txt"
with open(source_file, "w") as f:
    f.write('\n'.join([' '.join(doc) for doc in docs]))

### Shuyo Experiments

In [75]:
alpha, beta, gamma = 1, 0.500, 1
corpus = load_file(source_file)
vocab = Vocabulary(excluds_stopwords=False)
shuyo_docs = [vocab.doc_to_ids(doc) for doc in corpus]

In [ ]:
hdplda = HDPLDA(alpha, beta, gamma, shuyo_docs, vocab.size())

If Shuyo's code matches the original implementation, we should expect this to get 50-80 topics and a perplexity under 800. It's still running _slowly_. (This already ran 100 interations prior to this.

In [ ]:
while True:
    hdplda.inference()
    perplexity = hdplda.perplexity()
    print "- K=%d p=%f" % (len(hdplda.using_k)-1, perplexity)

- K=27 p=850.036583
- K=29 p=848.867273
- K=27 p=848.869948
- K=30 p=847.849667
- K=31 p=846.927765
- K=31 p=845.864101
- K=29 p=845.398586
- K=28 p=844.663386
- K=29 p=843.789580
- K=29 p=843.609830
- K=28 p=842.799414
- K=28 p=842.489442
- K=29 p=841.660747
- K=30 p=840.769906
- K=29 p=840.322523
- K=31 p=839.885603
- K=30 p=839.915356
- K=31 p=839.180617
- K=28 p=838.798009
- K=28 p=838.452270
- K=28 p=838.054395
- K=28 p=837.859003

Teh, et al, provide minimal information about their experiment with this data. The only mention of it is described below:

<img src="./images/teh-abstracts.png" width='600'>
<img src="./images/teh-figs.png" width='600'>

When we run Shuyo's code until the perplexity changes by only 0.1%, we get a similar perplexity to Teh.

## Label Permutation Tests

The HDP-LDA algorithm should be invariant under a variety of transformations of the input data. Thus, we can gain some confidence in Shuyo's implementation by verifying these invariant hold.

In [22]:
def generate_docs(num_docs, vocab_size, num_topics, doc_len_dist, alpha=0.5):
    docs = []
    topics = sp.stats.dirichlet([alpha for _ in range(vocab_size)]).rvs(num_topics)
    vocab = range(vocab_size)
    for doc in range(num_docs):
        doc_len = doc_len_dist()
        topic_index = np.random.choice(range(len(topics)))
        topic = topics[topic_index]
        docs.append(list(np.random.choice(vocab, size=doc_len, p=topic)))
    return docs
     
def rotate_labels(docs, vocab_size):
    new_docs = []
    for doc in docs:
        new_docs.append([(word + 1) % vocab_size for word in doc])
    return new_docs

def rotate_docs(docs, vocab_size):
    return docs[-1:] + docs[:-1]

def shuffle_words(docs, vocab_size):
    new_docs = []
    for doc in docs:
        new_doc = doc[:]
        shuffle(new_doc)
        new_docs.append(new_doc)
    return new_docs

def single_word(docs, vocab_size):
    new_docs = []
    for doc in docs:
        new_docs.append([docs[0][0] for _ in doc])
    return new_docs

def string_labels(docs, vocab_size):
    new_docs = []
    for doc in docs:
        new_docs.append([str(word) for word in doc])
    return new_docs

In [43]:

def trial(docs, vocab_size, alpha, beta, gamma, tol=0.001, max_steps=1000):
    hdplda = HDPLDA(alpha=alpha, beta=beta, gamma=gamma, docs=docs, V=vocab_size)
    perplexity = hdplda.perplexity()
    for _ in range(max_steps):
        hdplda.inference()
        new_perplexity = hdplda.perplexity()
        if abs(new_perplexity - perplexity) / (new_perplexity + perplexity) < tol:
            print ".",
            return {"perplexity": new_perplexity, "topics": len(hdplda.using_k)-1}
        perplexity = new_perplexity

def run_trials(docs, vocab_size, doc_manipulation=None, alpha=1, beta=.5, gamma=1, num_trials=50):
    if doc_manipulation is not None:
        print doc_manipulation.__name__,
        docs = doc_manipulation(docs, vocab_size)
    results = [trial(docs, vocab_size, alpha=1, beta=0.5, gamma=1) for _ in range(num_trials)]
    return results




In [44]:
vocab_size=5
docs = generate_docs(num_docs=50, vocab_size=vocab_size, num_topics=5, doc_len_dist=lambda: sp.stats.poisson(500).rvs())

In [45]:
results = {
    "baseline": run_trials(docs, vocab_size, None),
    "string_labels": run_trials(docs, vocab_size, string_labels),
    "shuffle_words": run_trials(docs, vocab_size, shuffle_words),
    "rotate_labels": run_trials(docs, vocab_size, rotate_labels),
    "rotate_docs": run_trials(docs, vocab_size, rotate_docs),
    "single_word": run_trials(docs, vocab_size, single_word),
    }
    

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . string_labels . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . shuffle_words . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . rotate_labels . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . rotate_docs . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . single_word . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


Perplexity and number of topics remains invariant under all transformations except the single_word transformation that actually changes the text.

In [62]:
pd.DataFrame({k: [l['perplexity'] for l in v] for k, v in results.iteritems()}).describe()

,baseline,rotate_docs,rotate_labels,shuffle_words,single_word,string_labels
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,3.101261,3.101178,3.101342,3.101319,1.000086,3.101484
std,0.000763,0.000515,0.000551,0.000541,0.000006,0.000682
min,3.099200,3.100121,3.099553,3.099596,1.000085,3.099717
25%,3.100883,3.100859,3.100984,3.101066,1.000085,3.101029
50%,3.101202,3.101238,3.101241,3.101404,1.000085,3.101489
75%,3.101669,3.101571,3.101609,3.101668,1.000086,3.102003
max,3.103261,3.102345,3.102566,3.102330,1.000125,3.102935


In [60]:
pd.DataFrame({k: [l['topics'] for l in v] for k, v in results.iteritems()}).describe()

,baseline,rotate_docs,rotate_labels,shuffle_words,single_word,string_labels
count,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000
mean,7.060000,6.90000,6.840000,7.120000,1.020000,7.120000
std,1.018402,1.05463,0.911603,0.872248,0.141421,0.848528
min,6.000000,6.00000,6.000000,6.000000,1.000000,6.000000
25%,6.000000,6.00000,6.000000,6.250000,1.000000,7.000000
50%,7.000000,7.00000,7.000000,7.000000,1.000000,7.000000
75%,8.000000,7.00000,7.000000,8.000000,1.000000,8.000000
max,10.000000,10.00000,10.000000,9.000000,2.000000,9.000000
